In [78]:
#8/8/24 Creating early fusion (not cold fusion) using morgan fingerprints and drugbank ids
#By doing this we get an amalgalm dataset including both drug target data and fingerprint data
#maybe better for prediction?

import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
#drugbank data
dbdatasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Targets/Drugbank/datasets/'
dbnamelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = df.drop(['Unnamed: 0'], axis = 1)
        dbdatasets.append(df)
        dbnamelist.append(filename[:-13])
#fingerprints
fpdatasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Fingerprints/'
fpnamelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' and 'organ' in filename and 'elopmental' not in filename and 'Reproductive' not in filename and 'iratory' not in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
#        df.drop(columns=['SMILES'], inplace=True)
        fpdatasets.append(df)
        fpnamelist.append(filename[:-13])
#raws
rawlist = []
index = []
directory = '/Users/james/Documents/Honours/Data/Raws/'
rawnames = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename and 'elopmental' not in filename and 'Reproductive' not in filename and 'iratory' not in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
#        df.drop(columns=['SMILES'], inplace=True)
        rawlist.append(df)
        rawnames.append(filename)

In [79]:
rawnames

['Cardiotoxicity_Cardiotoxicity-10.csv',
 'Carcinogenicity_Carcinogenicity.csv',
 'Endocrine Disruption_NR-aromatase.csv',
 'Hepatotoxicity_Hepatotoxicity.csv',
 'Endocrine Disruption_SR-p53.csv',
 'Endocrine Disruption_NR-AhR.csv',
 'Endocrine Disruption_NR-AR-LBD.csv',
 'Endocrine Disruption_SR-ARE.csv',
 'Cardiotoxicity_Cardiotoxicity-5.csv',
 'Endocrine Disruption_NR-ER.csv',
 'Cardiotoxicity_Cardiotoxicity-1.csv',
 'Endocrine Disruption_NR-ER-LBD.csv',
 'Endocrine Disruption_NR-PPAR-gamma.csv',
 'Endocrine Disruption_NR-AR.csv',
 'Mutagenicity_Ames Mutagenicity.csv',
 'Cardiotoxicity_Cardiotoxicity-30.csv',
 'Endocrine Disruption_SR-ATAD5.csv',
 'Endocrine Disruption_SR-HSE.csv',
 'Endocrine Disruption_SR-MMP.csv']

In [80]:
#links the fp->db and raw names
correlator = []
rawcorrel = []
for item in fpnamelist:
    loops = 0
    for name in dbnamelist:
        if item + '_' in name:
            correlator.append(loops)
        loops += 1
    loops = 0
    for name in rawnames:
        if item + '.' in name:
            rawcorrel.append(loops)
        loops += 1

In [67]:
rawcorrel

[16, 5, 3, 8, 12, 0, 18, 11, 1, 13, 9, 7, 6, 17, 2, 4, 10, 15]

In [81]:
#As dbdata is in inchikey and fp is in smiles need raw to link the two
dbdata = []
dbnames = []
for val in correlator:
    dbdata.append(dbdatasets[val])
    dbnames.append(dbnamelist[val])

rawdata = []
rawname = []
for val in rawcorrel:
    rawdata.append(rawlist[val])
    rawname.append(rawnames[val])

In [82]:
inchilist=[]
for df in rawdata:
    inchilist.append(list(df['InChIKey']))

In [83]:
#link inchikeys to smiles based on index 
#fingerprints are indexed same way as raw origin files

loops = 0
dbtofp = []
for df in dbdata:
    data = list(df['inchi'])
    index = []
    iter = 0
    for key in data:
        iter = 0
        seen = 0
        for item in inchilist[loops]:
            if key == item and seen == 0:
                index.append(iter)
                seen = 1
            iter += 1
    dbtofp.append(index)
    loops += 1
    

In [112]:
#create concatenated fp sets
concatfp = []
loops = 0
for df in dbtofp:
    temp = []
    for val in df:
        temp.append(fpdatasets[loops].loc[val])
    temp = pd.DataFrame(temp)
    temp.drop(columns=['SMILES', 'Toxicity_Value'], inplace=True)
    temp.reset_index(drop=True, inplace=True)
    temp = pd.concat([dbdata[loops], temp], axis=1)
    concatfp.append(temp)
    loops += 1

In [120]:
loops = 0
for df in concatfp:
    path = '/Users/james/Documents/Honours/Data/Fusion/'
    name = fpnamelist[loops] + '_fusion.csv'
    pathname = path + name
    df.to_csv(pathname)
    loops += 1

In [117]:
fpnamelist

['SR-ATAD5',
 'NR-AhR',
 'Hepatotoxicity',
 'Cardiotoxicity-5',
 'NR-PPAR-gamma',
 'Cardiotoxicity-10',
 'SR-MMP',
 'NR-ER-LBD',
 'Carcinogenicity',
 'NR-AR',
 'NR-ER',
 'SR-ARE',
 'NR-AR-LBD',
 'Ames Mutagenicity',
 'SR-HSE',
 'NR-aromatase',
 'SR-p53',
 'Cardiotoxicity-1',
 'Cardiotoxicity-30']

In [118]:
fpnamelist[0] +'_fusion'

'SR-ATAD5_fusion'